In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product


tf.random.set_seed(42)
# Function to build and train the model
def build_and_train_model(X_train, y_train_one_hot, X_val, y_val_one_hot, num_hidden_layers, num_nodes_values, learning_rate, activation_functions, num_epochs, batch_size, loss_function):
    model = models.Sequential()
    
    # Add input layer
    model.add(layers.InputLayer(input_shape=(2,)))
    
    # Add hidden layers
    for i in range(num_hidden_layers):
        num_nodes = num_nodes_values[i]
        activation_function = activation_functions[i]
        model.add(layers.Dense(num_nodes, activation=activation_function))
    
    # Add output layer
    model.add(layers.Dense(2, activation='softmax'))
    
    # Set optimizer and compile the model
    if learning_rate is not None:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam()  # Use default learning rate
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])
    
    # Train the model
    history = model.fit(X_train, y_train_one_hot, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, y_val_one_hot), verbose=0)
    
    return model, history


# Redefine parameters for the new task
num_hidden_layers_values = [2, 3]
num_nodes_values = {2: [16, 32], 3: [16, 32, 64]}
learning_rate_values = [0.01, 0.0001, None]  # None for default learning rate
num_epochs_values = [20, 30]
batch_size_values = [10, 50]
num_samples_values = [500, 1000, 2000]
# Define loss functions to test
loss_functions = ['categorical_crossentropy', 'mse']
# Define all possible activation functions
possible_activation_functions = ['relu', 'sigmoid', 'tanh']
# Generate all possible combinations of activation functions for each layer
activation_functions_combinations = list(product(possible_activation_functions, repeat=max(num_hidden_layers_values)))
# Create a dictionary to store results
results_dict = {}
# Get the total number of combinations
total_combinations = (
    len(num_samples_values)
    * len(num_hidden_layers_values)
    * len(learning_rate_values)
    * len(num_epochs_values)
    * len(batch_size_values)
    * len(list(product(possible_activation_functions, repeat=max(num_hidden_layers_values))))
    * len(loss_functions)
)
progress_counter = 0

# Create separate datasets for testing and validation
num_samples_test = 300
np.random.seed(123)
load_test = np.random.randint(0, 100, size=num_samples_test)
popularity_test = np.random.randint(0, 100, size=num_samples_test)
labels_test = np.where((load_test < 50) & (popularity_test > 50), 0, 1)
load_popularity_data_test = np.column_stack((load_test, popularity_test))

# Create a validation dataset
num_samples_val = 300
np.random.seed(456)
load_val = np.random.randint(0, 100, size=num_samples_val)
popularity_val = np.random.randint(0, 100, size=num_samples_val)
labels_val = np.where((load_val < 50) & (popularity_val > 50), 0, 1)
load_popularity_data_val = np.column_stack((load_val, popularity_val))

# Combine test and validation data for scaling
combined_data = np.vstack((load_popularity_data_test, load_popularity_data_val))

# Create the scaler and fit/transform the data
scaler = MinMaxScaler()
combined_data_scaled = scaler.fit_transform(combined_data)

# Separate the scaled data back into test and validation sets
load_popularity_data_test = combined_data_scaled[:num_samples_test, :]
load_popularity_data_val = combined_data_scaled[num_samples_test:, :]

X_test, y_test = load_popularity_data_test, labels_test
y_test_one_hot = to_categorical(y_test, num_classes=2)

X_val, y_val = load_popularity_data_val, labels_val
y_val_one_hot = to_categorical(y_val, num_classes=2)



# Iterate over different hyperparameter combinations
for num_samples in num_samples_values:
    print(f"Number of Samples: {num_samples}")
    # Generate random values for Load and Popularity
    np.random.seed(42)
    load = np.random.randint(0, 100, size=num_samples)
    popularity = np.random.randint(0, 100, size=num_samples)

    # Create labels
    labels = np.where((load < 50) & (popularity > 50), 0, 1)

    # Normalize the data
    scaler = MinMaxScaler()
    load_popularity_data = scaler.fit_transform(np.column_stack((load, popularity)))

    # Split into train set
    X_train, y_train = load_popularity_data, labels
    y_train_one_hot = to_categorical(y_train, num_classes=2)

# Counter for tracking progress

    for num_hidden_layers in num_hidden_layers_values:
        print(f"  Number of Hidden Layers: {num_hidden_layers}")
        print(f"  Number of Nodes: {num_nodes_values[num_hidden_layers]}")
        for learning_rate in learning_rate_values:
            print(f"      Learning Rate: {learning_rate}")
            for num_epochs in num_epochs_values:
                print(f"        Number of Epochs: {num_epochs}")
                for batch_size in batch_size_values:
                    print(f"          Batch Size: {batch_size}")
                    for activation_functions in product(possible_activation_functions, repeat=num_hidden_layers):
                        print(f"            Activation Functions: {activation_functions}")
                        for loss_function in loss_functions:
                            print(f"              Loss Function: {loss_function}")
                            key = (num_samples, num_hidden_layers, tuple(num_nodes_values[num_hidden_layers]), learning_rate, num_epochs, batch_size, activation_functions, loss_function)
                            model, history = build_and_train_model(X_train, y_train_one_hot, X_val, y_val_one_hot, num_hidden_layers, num_nodes_values[num_hidden_layers], learning_rate, activation_functions, num_epochs, batch_size, loss_function)
                             # Evaluate the model on the separate test set
                            _, accuracy = model.evaluate(X_test, y_test_one_hot, verbose=0)
                            # Save results, model, and evaluation accuracy
                            results_dict[key] = {'model': model, 'history': history, 'X_test': X_test, 'y_test_one_hot': y_test_one_hot, 'accuracy': accuracy}
                            # Update progress
                            progress_counter += 1
                            completion_percentage = (progress_counter / total_combinations) * 100
                            print(f"Progress: {completion_percentage:.2f}%")

Number of Samples: 500
  Number of Hidden Layers: 2
  Number of Nodes: [16, 32]
      Learning Rate: 0.01
        Number of Epochs: 20
          Batch Size: 10
            Activation Functions: ('relu', 'relu')
              Loss Function: categorical_crossentropy
Progress: 0.03%
              Loss Function: mse
Progress: 0.05%
            Activation Functions: ('relu', 'sigmoid')
              Loss Function: categorical_crossentropy
Progress: 0.08%
              Loss Function: mse
Progress: 0.10%
            Activation Functions: ('relu', 'tanh')
              Loss Function: categorical_crossentropy
Progress: 0.13%
              Loss Function: mse
Progress: 0.15%
            Activation Functions: ('sigmoid', 'relu')
              Loss Function: categorical_crossentropy
Progress: 0.18%
              Loss Function: mse
Progress: 0.21%
            Activation Functions: ('sigmoid', 'sigmoid')
              Loss Function: categorical_crossentropy
Progress: 0.23%
              Loss Function

In [ ]:
import os
# Directory to save the confusion matrix images
output_root_directory = "confusion_matrices"
os.makedirs(output_root_directory, exist_ok=True)

def plot_confusion_matrix_with_counts(matrix, title, filename,accuracy):
    plt.figure(figsize=(12, 7))
    sns.heatmap(matrix, annot=True, fmt='g', cmap='coolwarm',
                xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'],
                annot_kws={"color": "black"})
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            plt.text(j + 0.5, i + 0.5, f"{matrix[i, j]}", ha='center', va='center', color='black', fontsize=10)
    plt.title(f'{title}\nAccuracy: {accuracy:.4f}')
    plt.savefig(filename)
    plt.show()
    plt.close()  # Close the figure to prevent it from being displayed interactively

# Define the hyperparameter values
num_samples_values = [500, 1000, 2000]  # Replace with your actual hyperparameter values
# Initialize a dictionary to store training data and labels for each num_samples
train_data_dict = {}

# Iterate over different hyperparameter combinations
for num_samples in num_samples_values:
    print(f"Number of Samples: {num_samples}")
    
    # Generate random values for Load and Popularity
    np.random.seed(42)
    load = np.random.randint(0, 100, size=num_samples)
    popularity = np.random.randint(0, 100, size=num_samples)
    
    # Create labels
    labels = np.where((load < 50) & (popularity > 50), 0, 1)
    
    # Normalize the data
    scaler = MinMaxScaler()
    load_popularity_data = scaler.fit_transform(np.column_stack((load, popularity)))
    
    # Split into train set
    X_train, y_train = load_popularity_data, labels
    
    # One-hot encode the labels
    one_hot_labels = to_categorical(y_train, num_classes=2)
    
    # Save the training data and one-hot encoded labels to the dictionary
    train_data_dict[num_samples] = {'X_train': X_train, 'y_train': one_hot_labels}




# Print all results at the end
for key, result in results_dict.items():
    num_samples, num_hidden_layers, num_nodes, learning_rate, num_epochs, batch_size, activation_function, loss_function = key
    model = result['model']
    history = result['history']
    X_test = result['X_test']
    y_test_one_hot = result['y_test_one_hot']
    accuracy = result['accuracy']
    # Evaluate the model on the test set
    _, test_accuracy = model.evaluate(X_test, y_test_one_hot, verbose=0)
    # Make predictions on the test set
    y_pred_test = model.predict(X_test)
    y_pred_classes_test = np.argmax(y_pred_test, axis=1)
    # Create confusion matrix for test set
    conf_mat_test = confusion_matrix(y_test, y_pred_classes_test)
    # Make predictions on the training set
    # Access training data for the current num_samples from the dictionary
    X_train = train_data_dict[num_samples]['X_train']
    y_train = train_data_dict[num_samples]['y_train']
    # Make predictions on the training set
    y_pred_train = model.predict(X_train)
    y_pred_classes_train = np.argmax(y_pred_train, axis=1)
   # Evaluate the model on the training set
    _, train_accuracy = model.evaluate(X_train, y_train, verbose=0)

    # Convert one-hot encoded labels to binary format for the training set
    y_train_binary = np.argmax(y_train, axis=1)

    # Create confusion matrix for training set
    conf_mat_train = confusion_matrix(y_train_binary, y_pred_classes_train)

    print(f"\nResults for {key}:")
    print(f"Number of Samples: {num_samples}")
    print(f"Number of Hidden Layers: {num_hidden_layers}")
    print(f"Number of Nodes: {num_nodes}")
    print(f"Learning Rate: {learning_rate}")
    print(f"Number of Epochs: {num_epochs}")
    print(f"Batch Size: {batch_size}")
    print(f"Activation Function: {activation_function}")
    print(f"Loss Function: {loss_function}")
    print(f"Test Accuracy: {test_accuracy}")
    print(f"Confusion Matrix (Test Set):\n{conf_mat_test}")
    print(f"Confusion Matrix (Training Set):\n{conf_mat_train}")
    model_directory = os.path.join(output_root_directory, f"model_{key}")
    os.makedirs(model_directory, exist_ok=True)
    test_matrix_filename = "confusion_matrix_test.png"
    train_matrix_filename = "confusion_matrix_train.png"
    model_directory = os.path.join(output_root_directory, f"model_{key}")
    os.makedirs(model_directory, exist_ok=True)
    test_matrix_filename = os.path.join(model_directory, "confusion_matrix_test.png")
    train_matrix_filename = os.path.join(model_directory, "confusion_matrix_train.png")
    plot_confusion_matrix_with_counts(conf_mat_test, 'Confusion Matrix (Test Set)', test_matrix_filename,accuracy)
    plot_confusion_matrix_with_counts(conf_mat_train, 'Confusion Matrix (Training Set)', train_matrix_filename,train_accuracy)



In [11]:
import os



# Function to print best and worst model details without modifying the results_dict
def print_best_worst_models(results_dict, top_n=5):
    # Sort models based on accuracy
    sorted_models = sorted(results_dict.items(), key=lambda x: x[1]['accuracy'], reverse=True)
    # Directory to save the confusion matrix images
    output_root_directory = "Best_confusion_matrices"
    os.makedirs(output_root_directory, exist_ok=True)
    # Print top N best models
    print(f"Top {top_n} Best Models:")
    for i in range(min(top_n, len(sorted_models))):
        model_info = sorted_models[i]
        print(f"Accuracy: {model_info[1]['accuracy']:.4f}, Configuration: {model_info[0]}")
        print()

    # Print top N worst models
    output_root_directory = "worst_confusion_matrices"
    os.makedirs(output_root_directory, exist_ok=True)
    # Print top N worst models
    print(f"\nTop {top_n} Worst Models:")
    for i in range(max(0, len(sorted_models) - top_n), len(sorted_models)):
        model_info = sorted_models[i]
        print(f"Accuracy: {model_info[1]['accuracy']:.4f}, Configuration: {model_info[0]}")
        print()

# Print top 5 best and worst models
print_best_worst_models(results_dict, top_n=5)


Top 5 Best Models:
Accuracy: 1.0000, Configuration: (500, 3, (16, 32, 64), 0.01, 20, 50, ('tanh', 'tanh', 'relu'), 'mse')

Accuracy: 1.0000, Configuration: (500, 3, (16, 32, 64), 0.01, 30, 50, ('tanh', 'tanh', 'relu'), 'categorical_crossentropy')

Accuracy: 1.0000, Configuration: (1000, 3, (16, 32, 64), 0.01, 30, 50, ('tanh', 'relu', 'relu'), 'mse')

Accuracy: 1.0000, Configuration: (2000, 3, (16, 32, 64), 0.01, 30, 10, ('relu', 'tanh', 'sigmoid'), 'mse')

Accuracy: 0.9967, Configuration: (500, 2, (16, 32), 0.01, 20, 10, ('tanh', 'relu'), 'mse')


Top 5 Worst Models:
Accuracy: 0.7867, Configuration: (2000, 3, (16, 32, 64), 0.0001, 30, 50, ('tanh', 'sigmoid', 'sigmoid'), 'categorical_crossentropy')

Accuracy: 0.7867, Configuration: (2000, 3, (16, 32, 64), 0.0001, 30, 50, ('tanh', 'sigmoid', 'sigmoid'), 'mse')

Accuracy: 0.4433, Configuration: (500, 2, (16, 32), 0.0001, 30, 50, ('tanh', 'sigmoid'), 'mse')

Accuracy: 0.2133, Configuration: (500, 2, (16, 32), 0.0001, 20, 50, ('tanh', 'sigm

In [ ]:
import pickle

# Save the results_dict to a file
with open('results_dict.pkl', 'wb') as file:
    pickle.dump(results_dict, file)

# Load the results_dict from the file
with open('results_dict.pkl', 'rb') as file:
    loaded_results_dict = pickle.load(file)

# Now 'loaded_results_dict' contains the same data as the original 'results_dict'


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product

# Define the confusion matrix
conf_mat = np.array([[178, 72], [170, 80]])

def plot_confusion_matrix_with_counts(matrix):
    plt.figure(figsize=(12, 7))
    sns.heatmap(matrix, annot=True, fmt='g', cmap='coolwarm',
                xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'],
                annot_kws={"color": "black"})
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            plt.text(j + 0.5, i + 0.5, f"{matrix[i, j]}", ha='center', va='center', color='black', fontsize=10)
    #plt.savefig(filename)
    plt.show()
    plt.close() 

plot_confusion_matrix_with_counts(conf_mat)